In [1]:
from azureml.core import Workspace, Environment
from azureml.core.conda_dependencies import CondaDependencies

import plotly.graph_objects as go
import plotly.io as pio
import json

import json
import joblib
import pandas as pd
from azureml.core.model import Model
import os

In [1]:
# Connect to workspace
ws = Workspace.from_config()

# Create environment
env = Environment(name="churn-prediction-env")

# Add dependencies
conda_deps = CondaDependencies()
conda_deps.add_conda_package("scikit-learn")
conda_deps.add_conda_package("pandas")
conda_deps.add_conda_package("numpy")
conda_deps.add_pip_package("joblib")
conda_deps.add_pip_package("azureml-defaults")

# Add visualization libraries
conda_deps.add_conda_package("plotly")
conda_deps.add_conda_package("matplotlib")
conda_deps.add_conda_package("seaborn")

env.python.conda_dependencies = conda_deps

# Register workspace
env.register(workspace=ws)

{
    "assetId": "azureml://locations/polandcentral/workspaces/d5d742c6-a47b-4566-92f6-dd94aaf2a52c/environments/churn-prediction-env/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "churn-prediction-env",

In [4]:
# Creating score.py file
score_content = '''
import json
import joblib
import pandas as pd
from azureml.core.model import Model

predict_function = None

def init():
    global predict_function
    
    try:
        predict_function = joblib.load(Model.get_model_path('predict_function'))
        print("Model loaded successfully!")
        
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        raise

def run(raw_data):
    try:
        data = json.loads(raw_data)
        
        if isinstance(data, dict):
            df = pd.DataFrame([data])
        else:
            df = pd.DataFrame(data)
        
        # Download path to pipeline
        pipeline_path = Model.get_model_path('churn_model_pipeline')
        
        # Use predict_function
        result = predict_function(df, pipeline_path)
        
        return json.dumps(result)
        
    except Exception as e:
        error_msg = f"Error during prediction: {str(e)}"
        return json.dumps({"error": error_msg})
'''

# Save score.py
with open('score.py', 'w') as f:
    f.write(score_content)
print("score.py created")

score.py created
